In [1]:
from Model import *
import torch.nn as nn
import pandas as pd

#get the data
data = pd.read_csv("Data/dataset.csv")
train = data['queue'].values
labels = data['label'].values
tokenized = tokenize(train)

/var/folders/5m/5bz8j_1n6bv7tpcnz6sjyc280000gn/T/ipykernel_9465/4160086905.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
#model hyperparameters 
ntokens = 65536  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in ``nn.TransformerEncoder``
nlayers = 3  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
nhead = 2  # number of heads in ``nn.MultiheadAttention``
dropout = 0.2  # dropout probability

#define the model 
transformer = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(transformer.parameters())

#training loop
for i in range(3):
    transformer.train()
    for j in range(10):
        optimizer.zero_grad()
        input = tokenized[j*128:j*128+128]
        output = transformer(input)
        loss = criterion(output, torch.Tensor(labels[j*128:j*128+128]).reshape(-1, 1))

        loss.backward()
        optimizer.step()
        print(loss)
    print("epoch finished")